In [1]:
!pip install scimath

     |████████████████████████████████| 89 kB 594 kB/s eta 0:00:01
     |████████████████████████████████| 567 kB 207 kB/s eta 0:00:01
  Created wheel for scimath: filename=scimath-4.2.0-cp37-cp37m-linux_x86_64.whl size=162027 sha256=6c4809c6e1c817a96bddfe45dd16a9c73d4b590c3aab7e314f69fa2f1601d546
  Stored in directory: /media/linstore/symbolic_links/home_mazur_cache/pip/wheels/dd/98/dd/58838adeac022e14b81125973d20a9a3395c300402ecc48c8d
  Created wheel for traits: filename=traits-6.1.1-cp37-cp37m-linux_x86_64.whl size=538450 sha256=8431eab04f969e6f563a8cdd24f78746b8c7bc84b2c5e68166c2b39b5e4f38fc
  Stored in directory: /media/linstore/symbolic_links/home_mazur_cache/pip/wheels/8b/64/2c/b5f981e926c21043db05140e1b6b524c655e894e4c3d72ed65
Successfully built scimath traits


In [23]:
from scimath.units.length import meters as m
from scimath.units.length import kilometers as km
from scimath.units.length import light_year as ly
from scimath.units.mass import kilograms as kg
from scimath.units.time import seconds as s
import numpy as np

In [12]:
G = 6.674e-11 * m**3 / kg / s**2
c = 299792458 * m / s
solar_mass = 1.98847e30 * kg

In [18]:
def v_escape_solar(departure_distance):
    v = (2 * G * solar_mass / departure_distance)**0.5
    return v

print(v_escape_solar(150e6 * km))

42065.10633133674*m*s**-1


In [26]:
class Starship:
    def __init__(self,
                 payload_mass,
                 fuel_mass,
                 initial_velocity = 0 * m / s,
                 initial_position = 0 * km,
                 initial_time = 0 * s,
                 exhaust_velocity = 500 * km / s,
                 destination_distance = 4.244 * ly
                 ):
        self.payload_mass = payload_mass
        self.fuel_mass = fuel_mass
        self.velocity = initial_velocity
        self.position = initial_position
        self.time = initial_time
        self.exhaust_velocity = exhaust_velocity
        self.history = list()
        self.log_entry()
        
    def log_entry(self):
        new_log = {'time': self.time,
                   'distance': self.distance,
                   'velocity': self.velocity,
                   'fuel_mass': self.fuel_mass}
        self.history.append(new_log)
        
    def total_mass(self):
        return self.payload_mass + self.fuel_mass
        
    def accelerate(self,
                   target_velocity = 0 * km / s, 
                   fuel_mass = None,
                   decelerate = False):
        if fuel_mass is not None:
            if fuel_mass > self.fuel_mass:
                raise ValueError(f"Not enough fuel for this maneuver. Requested {fuel_mass} of {self.fuel_mass}.")
            delta_v = self.exhaust_velocity * np.log(self.total_mass / (self.total_mass - fuel_mass))
            if decelerate:
                self.velocity += delta_v
            else:
                self.velocity += delta_v
            self.fuel_mass -= fuel_mass
        
        else:
            final_mass = self.total_mass() * np.exp(-1 * np.abs(self.velocity - target_velocity) / self.exhaust_velocity)
            delta_fuel_mass = self.total_mass() - final_mass
            self.fuel_mass -= delta_fuel_mass
            if self.fuel_mass < 0:
                raise ValueError(f"Note enough fuel for this maneuver. Requested {delta_fuel_mass} of {self.fuel_mass + delta_fuel_mass}.")
            
            self.velocity = target_velocity
            
        self.log_entry()
            
        return self.velocity
    
    def cruise(self, distance):
        self.position += distance
        self.time += distance / self.velocity
        self.log_entry()
        
    def print_history(self):
        print(self.history)
        
    def plot_history(self):
        pass
    
        